In [16]:
# A 165 LINE TOPOLOGY OPTIMIZATION CODE BY NIELS AAGE AND VILLADS EGEDE JOHANSEN, JANUARY 2013
from __future__ import division
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve
from matplotlib import colors
import matplotlib.pyplot as plt
# MAIN DRIVER
def main(nelx,nely,volfrac,penal,rmin,ft, maxiter, minchange, f, fixed, shouldplot=True):
	Emin=1e-9
	Emax=1.0
	# dofs:
	ndof = 2*(nelx+1)*(nely+1)
	# Allocate design variables (as array), initialize and allocate sens.
	x=volfrac * np.ones(nely*nelx,dtype=float)
	xold=x.copy()
	xPhys=x.copy()
	g=0 # must be initialized to use the NGuyen/Paulino OC approach
	dc=np.zeros((nely,nelx), dtype=float)
	# FE: Build the index vectors for the for coo matrix format.
	KE=lk()
	edofMat=np.zeros((nelx*nely,8),dtype=int)
	for elx in range(nelx):
		for ely in range(nely):
			el = ely+elx*nely
			n1=(nely+1)*elx+ely
			n2=(nely+1)*(elx+1)+ely
			edofMat[el,:]=np.array([2*n1+2, 2*n1+3, 2*n2+2, 2*n2+3,2*n2, 2*n2+1, 2*n1, 2*n1+1])
	# Construct the index pointers for the coo format
	iK = np.kron(edofMat,np.ones((8,1))).flatten()
	jK = np.kron(edofMat,np.ones((1,8))).flatten()    
	# Filter: Build (and assemble) the index+data vectors for the coo matrix format
	nfilter=int(nelx*nely*((2*(np.ceil(rmin)-1)+1)**2))
	iH = np.zeros(nfilter)
	jH = np.zeros(nfilter)
	sH = np.zeros(nfilter)
	cc=0
	for i in range(nelx):
		for j in range(nely):
			row=i*nely+j
			kk1=int(np.maximum(i-(np.ceil(rmin)-1),0))
			kk2=int(np.minimum(i+np.ceil(rmin),nelx))
			ll1=int(np.maximum(j-(np.ceil(rmin)-1),0))
			ll2=int(np.minimum(j+np.ceil(rmin),nely))
			for k in range(kk1,kk2):
				for l in range(ll1,ll2):
					col=k*nely+l
					fac=rmin-np.sqrt(((i-k)*(i-k)+(j-l)*(j-l)))
					iH[cc]=row
					jH[cc]=col
					sH[cc]=np.maximum(0.0,fac)
					cc=cc+1
	# Finalize assembly and convert to csc format
	H=coo_matrix((sH,(iH,jH)),shape=(nelx*nely,nelx*nely)).tocsc()	
	Hs=H.sum(1)
	# BC's and support
	dofs=np.arange(2*(nelx+1)*(nely+1)) #Liste de chacuns des éléments
	free=np.setdiff1d(dofs,fixed)
	# Solution and RHS vectors
	u=np.zeros((ndof,1))
	# Initialize plot and plot the initial design
   	# Set loop counter and gradient vectors 
	loop=0
	change=1
	dv = np.ones(nely*nelx)
	dc = np.ones(nely*nelx)
	ce = np.ones(nely*nelx)
	while change>minchange and loop<maxiter:
		loop=loop+1
		#print(loop, end=" ")
		# Setup and solve FE problem
		sK=((KE.flatten()[np.newaxis]).T*(Emin+(xPhys)**penal*(Emax-Emin))).flatten(order='F')
		K = coo_matrix((sK,(iK,jK)),shape=(ndof,ndof)).tocsc()
		# Remove constrained dofs from matrix
		K = K[free,:][:,free]
		# Solve system 
		u[free,0]=spsolve(K,f[free,0])    
		# Objective and sensitivity
		ce[:] = (np.dot(u[edofMat].reshape(nelx*nely,8),KE) * u[edofMat].reshape(nelx*nely,8) ).sum(1)
		obj=( (Emin+xPhys**penal*(Emax-Emin))*ce ).sum()
		dc[:]=(-penal*xPhys**(penal-1)*(Emax-Emin))*ce
		dv[:] = np.ones(nely*nelx)
		# Sensitivity filtering:
		if ft==0:
			dc[:] = np.asarray((H*(x*dc))[np.newaxis].T/Hs)[:,0] / np.maximum(0.001,x)
		elif ft==1:
			dc[:] = np.asarray(H*(dc[np.newaxis].T/Hs))[:,0]
			dv[:] = np.asarray(H*(dv[np.newaxis].T/Hs))[:,0]
		# Optimality criteria
		xold[:]=x
		(x[:],g)=oc(nelx,nely,x,volfrac,dc,dv,g)
		# Filter design variables
		if ft==0:   xPhys[:]=x
		elif ft==1:	xPhys[:]=np.asarray(H*x[np.newaxis].T/Hs)[:,0]
		# Compute the change by the inf. norm
		change=np.linalg.norm(x.reshape(nelx*nely,1)-xold.reshape(nelx*nely,1),np.inf)
		# Write iteration history to screen (req. Python 2.6 or newer)
		"""print("it.: {0} , obj.: {1:.3f} Vol.: {2:.3f}, ch.: {3:.3f}".format(\
					loop,obj,(g+volfrac*nelx*nely)/(nelx*nely),change))"""
	# Make sure the plot stays and that the shell remains
	xPhys_2d = xPhys.reshape((nelx, nely))
	xPhys_2d = np.interp(xPhys_2d, (xPhys_2d.min(), xPhys_2d.max()), (0, 255)).astype(int)
	color_grid = [[(val, val, val) for val in row] for row in xPhys_2d]
	return color_grid
#element stiffness matrix
def lk():
	E=1
	nu=0.3
	k=np.array([1/2-nu/6,1/8+nu/8,-1/4-nu/12,-1/8+3*nu/8,-1/4+nu/12,-1/8-nu/8,nu/6,1/8-3*nu/8])
	KE = E/(1-nu**2)*np.array([ [k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
	[k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
	[k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
	[k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
	[k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
	[k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
	[k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
	[k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]] ]);
	return (KE)
# Optimality criterion
def oc(nelx,nely,x,volfrac,dc,dv,g):
	l1=0
	l2=1e9
	move=0.2
	# reshape to perform vector operations
	xnew=np.zeros(nelx*nely)
	while (l2-l1)/(l1+l2)>1e-3:
		lmid=0.5*(l2+l1)
		xnew[:]= np.maximum(0.0,np.maximum(x-move,np.minimum(1.0,np.minimum(x+move,x*np.sqrt(-dc/dv/lmid)))))
		gt=g+np.sum((dv*(xnew-x)))
		if gt>0 :
			l1=lmid
		else:
			l2=lmid
	return (xnew,gt)

In [17]:
nelx=90
nely=30
volfrac=0.4
rmin=3
penal=3.0
ft=1 # ft==0 -> sens, ft==1 -> dens
maxiter=1000
minchange=0.01
dofs=np.arange(2*(nelx+1)*(nely+1))
ndof = 2*(nelx+1)*(nely+1)
f=np.zeros((ndof,1))
volfrac=0.4
fixed=np.array([])

In [18]:
import pygame
import sys

# Set the dimensions
CELL_SIZE = 10
WIDTH, HEIGHT = 90, 30  # The grid size
BUTTON_WIDTH, BUTTON_HEIGHT = 140, 40  # The button size
SCREEN_WIDTH, SCREEN_HEIGHT = WIDTH * CELL_SIZE, HEIGHT * CELL_SIZE + BUTTON_HEIGHT*6
MARGIN = 20  # Margin between buttons

# Set the colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
MAGENTA = (255, 0, 255)
CYAN = (0, 255, 255)
BLACK = (0, 0, 0)  # Color for grid lines and button outlines
GRAY = (200, 200, 200)  # Color for button background

# Initialize the game
pygame.init()

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
screen.fill(WHITE)
colors = []
colors_calculated = False
# Initialize the grid with all white cells
grid = [[WHITE for _ in range(WIDTH)] for _ in range(HEIGHT)]
optimizing = False
# Set the initial drawing color to red
draw_color = RED

# Set the font
font = pygame.font.Font(None, 24)

# Define the buttons
buttons = {
    "red": pygame.Rect(MARGIN, HEIGHT * CELL_SIZE + MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
    "green": pygame.Rect(MARGIN + BUTTON_WIDTH + MARGIN, HEIGHT * CELL_SIZE + MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
    "blue": pygame.Rect(MARGIN + 2 * (BUTTON_WIDTH + MARGIN), HEIGHT * CELL_SIZE + MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
    "yellow": pygame.Rect(MARGIN, HEIGHT * CELL_SIZE + MARGIN + BUTTON_HEIGHT + MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
    "magenta": pygame.Rect(MARGIN + BUTTON_WIDTH + MARGIN, HEIGHT * CELL_SIZE + MARGIN + BUTTON_HEIGHT + MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
    "cyan": pygame.Rect(MARGIN + 2 * (BUTTON_WIDTH + MARGIN), HEIGHT * CELL_SIZE + MARGIN + BUTTON_HEIGHT + MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
    "black": pygame.Rect(MARGIN, HEIGHT * CELL_SIZE + 2*(MARGIN + BUTTON_HEIGHT)+MARGIN, BUTTON_WIDTH, BUTTON_HEIGHT),
}

# Define button colors and text
button_colors = { "red": RED, "green": GREEN, "blue": BLUE, "yellow": YELLOW, "magenta": MAGENTA, "cyan": CYAN, "black": BLACK }
button_texts = { 
    "red": font.render('Fy = +1', True, RED), 
    "green": font.render('Fixer sur X', True, GREEN), 
    "blue": font.render('Fixer sur Y', True, BLUE),
    "yellow": font.render('Fy = -1', True, YELLOW),
    "magenta": font.render('Fx = +1', True, MAGENTA),
    "cyan": font.render('Fx = -1', True, CYAN),
    "black": font.render('Optimiser', True, BLACK),
}

running = True
while running:
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            i = x // CELL_SIZE
            j = y // CELL_SIZE

            # Check if the click was on one of the color buttons
            for button, rect in buttons.items():
                if rect.collidepoint(x, y):
                    draw_color = button_colors[button]
                    if draw_color == BLACK:
                        optimizing = True
            # Check if the click was on the grid
            if j < HEIGHT:
                grid[j][i] = draw_color
                if draw_color == GREEN:
                    fixed=np.union1d(fixed,np.array([2*(i+1)*(j+1)]))
                elif draw_color == BLUE:
                    fixed=np.union1d(fixed,np.array([2*(i+2)*(j+2)-1]))
                elif draw_color == YELLOW:
                    f[2*i+1, 0]=1
                elif draw_color == MAGENTA:
                    f[2*j, 0]=1
    if not(optimizing):
        # Draw the grid
        for j, row in enumerate(grid):
            for i, col in enumerate(row):
                pygame.draw.rect(screen, col, (i * CELL_SIZE, j * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                pygame.draw.rect(screen, GRAY, (i * CELL_SIZE, j * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)  # Draw grid lines

        # Draw the color buttons
        for button, rect in buttons.items():
            pygame.draw.rect(screen, GRAY, rect)  # Draw button background
            pygame.draw.rect(screen, BLACK, rect, 2)  # Draw button outline
            screen.blit(button_texts[button], rect.move(20, 10))  # Draw button text
    else:
        if not(colors_calculated):
            colors = main(nelx,nely,volfrac,penal,rmin,ft, maxiter, minchange, f, fixed)
            colors_calculated = True
        for x in range(0, WIDTH):
            for y in range(0, HEIGHT):
                r, g, b = colors[x][y]
                r = 255-r
                g = 255-g
                b = 255-b
                pygame.draw.rect(screen, (r, g, b), (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                pygame.draw.rect(screen, GRAY, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
        
    # Update the display
    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 